In [66]:
!pip install tensorflow

In [67]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import BatchNormalization


In [68]:
import numpy as np 
from keras.layers import LeakyReLU
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
# from tensorflow.keras import models
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import BatchNormalization

In [69]:
def build_generator(z_dim):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(7*7*256, input_dim=z_dim))
    model.add(keras.layers.Reshape((7, 7, 256)))
    model.add(keras.layers.Conv2DTranspose(filters=128, kernel_size=(3,3), strides=(2, 2), padding="same"))
    model.add(keras.layers.BatchNormalization())   
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Conv2DTranspose(filters=64, kernel_size=(3,3), strides=(1, 1), padding="same"))
    model.add(keras.layers.BatchNormalization())   
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Conv2DTranspose(filters=1, kernel_size=(3,3),strides=(2, 2),activation="tanh", padding="same"))
   
    return model

In [70]:
model=build_generator(100)
model.summary()


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 12544)             1266944   
                                                                 
 reshape_6 (Reshape)         (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose_18 (Conv2D  (None, 14, 14, 128)      295040    
 Transpose)                                                      
                                                                 
 batch_normalization_24 (Bat  (None, 14, 14, 128)      512       
 chNormalization)                                                
                                                                 
 leaky_re_lu_30 (LeakyReLU)  (None, 14, 14, 128)       0         
                                                                 
 conv2d_transpose_19 (Conv2D  (None, 14, 14, 64)     

In [71]:
def build_disciminator():
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(filters=32,kernel_size=(3, 3),  strides=(2, 2),  padding='same', input_shape=(28, 28, 1))) 
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Conv2D(filters=64,kernel_size=(3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())   
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Conv2D(filters=128,kernel_size=(3, 3), strides=(2, 2)))
    model.add(keras.layers.BatchNormalization())   
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    return model

In [72]:
model=build_disciminator()
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu_32 (LeakyReLU)  (None, 14, 14, 32)        0         
                                                                 
 conv2d_19 (Conv2D)          (None, 7, 7, 64)          18496     
                                                                 
 batch_normalization_26 (Bat  (None, 7, 7, 64)         256       
 chNormalization)                                                
                                                                 
 leaky_re_lu_33 (LeakyReLU)  (None, 7, 7, 64)          0         
                                                                 
 conv2d_20 (Conv2D)          (None, 3, 3, 128)         73856     
                                                     

In [73]:
def build_Gan(generator, discriminator):
  discriminator.trainable=False
  model = keras.models.Sequential()
  model.add(generator)
  model.add(discriminator)
  return model


In [74]:
generator=build_generator(100)
discriminator=build_disciminator()
discriminator.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.binary_crossentropy, metrics=[keras.metrics.binary_accuracy])
disciminator.trainable=False
gan=build_Gan(generator, discriminator)
gan.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.binary_crossentropy)


In [75]:
gan.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_19 (Sequential)  (None, 28, 28, 1)         1637121   
                                                                 
 sequential_20 (Sequential)  (None, 1)                 94593     
                                                                 
Total params: 1,731,714
Trainable params: 1,636,737
Non-trainable params: 94,977
_________________________________________________________________


In [76]:
from keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X.shape


(60000, 28, 28)

In [ ]:
batch_size = 128
real_label = np.ones((batch_size,1))
fake_label = np.zeros((batch_size,1))
for i in range(1000):
  indices = np.random.randint(0, train_X.shape[0], batch_size)
  x_real = train_X[indices]
  z_dim=100
  noise=np.random.normal(0,1,(batch_size, z_dim))
  x_fake=generator.predict(noise)

  loss_real = discriminator.train_on_batch(x_real, real_label)
  loss_fake = discriminator.train_on_batch(x_fake, fake_label)
  loss_general=0.5*(np.add(loss_real,loss_fake))
  if i %100 ==0:
    print(f"loss_general: {loss_general}")

  indices = np.random.randint(0, train_X.shape[0], batch_size)
  x_real = train_X[indices]
  z_dim=100
  noise=np.random.normal(0,1,(batch_size, z_dim))
  x_fake=generator.predict(noise)
  loss_fake =gan.train_on_batch(noise,real_label)
  if i %100 ==0:
    print(f"loss_fake: {loss_fake}")


loss_general: [8.15589621e-04 1.00000000e+00]
loss_fake: 0.4837302565574646
loss_general: [4.18691357e-04 1.00000000e+00]
loss_fake: 0.2513490319252014
loss_general: [2.54636499e-04 1.00000000e+00]
loss_fake: 0.1433069109916687
loss_general: [2.85503804e-04 1.00000000e+00]
loss_fake: 0.10427699983119965
